In [ ]:
import keras, os
from keras import Input
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.vgg19 import VGG19
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix,f1_score,recall_score,precision_score
from keras.models import Model, load_model
from sklearn.metrics.pairwise import cosine_similarity
import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.image import load_img

In [ ]:
num_classes = 64
batch_size = 64
img_height = 224
img_width = 224
nb_epochs = 10

datagen = ImageDataGenerator()  # set validation split
arabic_data = datagen.flow_from_directory(
    'arabic',
    target_size=(224, 224),
    batch_size=batch_size,shuffle=False)  # set as training data
class_indices_flipped = {v: k for k, v in arabic_data.class_indices.items()}
# Step 1 Train the VGG Model to classify the manuscript
arabic_data.class_indices

In [ ]:
def build_model_local():
    base_model = VGG19(weights='imagenet',
                       include_top=False)  # imports the Vgg19 model and discards the last 1000 neuron layer.
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='sigmoid')(
        x)  # we add dense layers so that the model can learn more complex functions and classify for better results.
    x = Dense(1024, activation='sigmoid')(x)  # dense layer 2
    x = Dense(512, activation='sigmoid')(x)  # dense layer 3
    preds = Dense(64, activation='softmax')(x)  # final layer with softmax activation

    model = Model(inputs=base_model.input, outputs=preds)
    return model


model = build_model_local()
model.compile(optimizer='Adam', loss='weighted_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# If we need to use a penultimate layer
new_model = Model(model.inputs, model.layers[-2].output)
new_model.compile(optimizer='Adam', loss='weighted_categorical_crossentropy', metrics=['accuracy'])
new_model.summary()

In [ ]:
X = new_model.predict_generator(arabic_data, len(arabic_data))

In [ ]:
X.shape # 8368,512
Y= arabic_data.classes

#Number of similar pages to retrieve
K = 10 

In [ ]:
filenames = np.array(arabic_data.filenames)
filepaths = np.array(arabic_data.filepaths)
data = {'filenames': filenames, 'filepaths' : filepaths, 'manuscriptID': Y}
new_df = pd.DataFrame(data)

new_df


In [ ]:
def get_input(path):
    image = load_img(path, target_size=(224, 224))
    return np.array(image).reshape(1,224,224,3)

In [ ]:
start_time = datetime.datetime.now().timestamp()

total_accuracy = 0
Y_pred_list = []

count = 0

for key,image_path in new_df['filepaths'].iteritems():
    #Get the predicted feature vector for the given image
    pred_feature_vec = new_model.predict(get_input(image_path))
    
    #Find the cosine similarity array based on all the feature vectors 
    #stored in X
    similarity_array = cosine_similarity(pred_feature_vec, X)[0]
    
    #Get top K indices 
    indices = similarity_array.argsort()[-K:][::-1]
    
    true_ID = new_df['manuscriptID'].loc[key]
    total_pages = new_df[ new_df['manuscriptID'] == true_ID]['filenames'].count()
    predicted_arr = new_df['manuscriptID'].loc[indices].values
    
    Y_pred_list.append(predicted_arr[0])
    
    #Number of correct predictions out of K
    found = np.count_nonzero(predicted_arr == true_ID)
    #print(indices)    
    if total_pages >= K:
        total_accuracy +=  found/K
    else:
        #total_pages is less than K
        total_accuracy += found/total_pages
    
    count += 1
    
    if count > 0 and count % 100 == 0:
        print("Done ", count)
        print("Accuracy so far %g %%" % (total_accuracy/count * 100))
       
end_time =  datetime.datetime.now().timestamp()
total_retrieval_time = end_time - start_time #In Seconds
print("Total retrieval time %g seconds" % total_retrieval_time)

mean_accuracy = total_accuracy/new_df.shape[0]
print("\nThe mean accuracy for top %d images is %g %%" % (K, mean_accuracy*100))

In [ ]:
confusion_matrix(Y, Y_pred_list).shape

In [ ]:
#Plot the confusion matrix
#Note that Y_pred_list contains the top prediction
sn.heatmap(confusion_matrix(Y, Y_pred_list), cmap=plt.cm.Blues, annot=True)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Manuscript ID')
plt.ylabel('True Manuscript ID')
plt.show()

The confusion matrix above does not have a single false positive. So the top predicted class is always predicted correctly.

In [ ]:
f1_score(Y, Y_pred_list,average='micro')

In [ ]:
recall_score(Y, Y_pred_list,average='micro')

In [ ]:
precision_score(Y, Y_pred_list,average='micro')

In [ ]:
query_image_path = "*/arabic/3/DSC00009.JPG"

In [ ]:
start_time = datetime.datetime.now().timestamp()
output = new_model.predict(get_input(query_image_path)) # VGG Model - 512
similarity_array = cosine_similarity(output, X)[0]
 
#Get top K indices 
indices = similarity_array.argsort()[-10:][::-1]
end_time =  datetime.datetime.now().timestamp()

total_retrieval_time = end_time - start_time #In Seconds
total_retrieval_time

In [ ]:
print(filenames[indices])
similarity_array[similarity_array.argsort()[-10:][::-1]]